<a href="https://colab.research.google.com/github/bskang8/BS-Fineway/blob/master/MKBHD_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain -q
!pip install openai==0.28 -q

In [4]:
!pip install chromadb -q
!pip install chromadb tiktoken -q

In [10]:
import os

from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [6]:
os.environ["OPENAI_API_KEY"] = ""

##Text 파일을 chroma DB로 vector화 하기

In [7]:
## 1. Text 불러들이기
text_path = '/content/drive/MyDrive/IT_Q_n_A/text_file/MKBHD_script.txt'
with open(text_path , "r") as f:
    text = f.read()


## 2. Text chunk화
def text_splitter(text, chunk_size, chunk_overlap):
  """
  Text를 vector화 하기 위하여 작은 chunk 단위로 분리하는 함수.
  단, chroma DB로 만들기 위하여 각 chunk는 Document로 wrapping 해야 함

  text: 분리하기 위한 text 전체
  chunk_size: 분리를 위한 크기 설정 (현재는 text길이 단우, 향후 token 길이단위로 변경 가)
  chunk_overlap: 연결되는 chunk의 겹치는 크기 (text 길이 단위)
  """
  chunk_list = []
  chunk_idx = 0
  while True:
    st_posi = max(chunk_size * chunk_idx - chunk_overlap, 0)
    if st_posi >= len(text):
      break
    end_posi = min(chunk_size * (chunk_idx+1), len(text))

    chunk = text[st_posi:end_posi]
    chunk_list.append(Document(page_content=chunk,
                              metadata={
                                  'part' : chunk_idx+1
                              }
    ))
    chunk_idx += 1
  return chunk_list
chunk_list = text_splitter(text=text, chunk_size=1000, chunk_overlap=20)


## 3. Chunk를 embedding vector화 (tokenizer 및 encoding 모델 필요, openai api는 한번에 제공)
persist_directory = '/content/drive/MyDrive/IT_Q_n_A/embedding_vectors/MKBHD_v1'
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=chunk_list, embedding=embeddings, persist_directory=persist_directory)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


## Chroma DB를 LLM에 연결하여 Q&A 시스템을 구축

In [11]:
## 1. Vector DB를 로딩 후 retriever를 위한 시스템 구축
embeddings = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
doc_retriever = vectordb.as_retriever()


## 2.LLM 모델 불러오기
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")


## 3. IT_Q_n_A 시스템 구축하기
IT_qna = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_retriever)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


## Q&A 구동 test

In [14]:
answer = IT_qna.run("Please tell me what IT products are being introduced")
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprec

Based on the given context, the following IT products are being introduced:

1. M2 Pro and M2 Max: These are new products announced earlier this year, along with the new MacBook Pro and Mac Mini. Users have been amazed by their performance.

2. Mac Studio Max: This is an entirely new product designed specifically for professionals, offering breakthrough performance and expansion options.

3. Apple silicon: Apple is bringing its silicon technology to more Pro products, including Mac Pro and potentially iMac Pro.

4. AR platform: Apple is introducing a new AR platform, along with a revolutionary new product called Vision Pro, which delivers advanced AR capabilities.

Please note that the given context is not very clear, and some information may be speculative or incomplete.
